In [1]:
from datetime import datetime
import datetime

from tqdm import tqdm
from datetime import date
import pandas as pd
import numpy as np
import random as rdm


In [60]:
df_csv = pd.read_csv('bank_transactions.csv')

In [61]:
#drop some column and renamed
df_ = df_csv[['CustomerID','CustGender','CustomerDOB','CustAccountBalance','TransactionAmount (INR)']]
df_.columns= ['ID','Gender','DOB','CABalance','TransA']

In [48]:
df_.head()

,ID,Gender,DOB,CABalance,TransA
0,C5841053,F,10/1/94,17819.05,25.0
1,C2142763,M,4/4/57,2270.69,27999.0
2,C4417068,F,26/11/96,17874.44,459.0
3,C5342380,F,14/9/73,866503.21,2060.0
4,C9031234,F,24/3/88,6714.43,1762.5


In [34]:
df_.DOB=pd.to_datetime(df_.DOB)

/var/folders/cg/glf2wdrs1_q18pg9grzgpjqr0000gp/T/ipykernel_51737/1441423561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.DOB=pd.to_datetime(df_.DOB)


To build the characteristic matrix, we will create some membership intervals.
The number of intervals and the way to define them has been changed several times. The idea was not to leave practically empty intervals, but to have a certain number of them and then reduce it in the permutation phase.

In [6]:
def age(x):
    today = date.today()
    if today.year <= x.year or today.year - x.year > 100:
        return 0
    else:
        age = today.year - x.year - ((today.month, today.day) < (x.month, x.day))
    return age

def age_class(x):
    if x <= 15: return 0
    elif x > 15 and x <=25: return 1
    elif x > 25 and x <=35: return 2
    elif x > 35 and x <=45: return 3
    elif x > 45: return 4


In [7]:
CABmean = df_.CABalance.mean()
CABmin = df_.CABalance.min()
CABmax = df_.CABalance.max()
bins_CAB = np.linspace(CABmin, 2*CABmean, 50)
def CAB_class(x):
    clas=0
    if x > 2*CABmean:
        clas=50
    else:
        for i in range(len(bins_CAB)-1):
            if x >= bins_CAB[i] and x < bins_CAB[i+1]:
                clas=i
                break
    return clas
    

In [8]:
Trans_mean = df_.TransA.mean()
Trans_min = df_.TransA.min()
Trans_max = df_.TransA.max()
bins_T = np.linspace(Trans_min, 2*Trans_mean, 50)
def Trans_class(x):
    clas=0
    if x > 2*Trans_mean:
        clas=50
    else:
        for i in range(len(bins_T)-1):
            if x >= bins_T[i] and x < bins_T[i+1]:
                clas=i
                break
    return clas
    

In [9]:
new_df=pd.DataFrame()
new_df['ID']=df_.ID  
new_df['CAB']=df_.CABalance.apply(lambda x: CAB_class(x) )  # 50 class
new_df['Trans']=df_.TransA.apply(lambda x: Trans_class(x) )  # 50 class
new_df['Age']=df_.DOB.apply(lambda x: age_class(age(x)) )  # 5 class
new_df['Gender']=df_.Gender.apply(lambda x: 0 if x=='M' else 1)  # 2 class

In [10]:
new_df=new_df.dropna()
new_df.Age.astype(int)

0          2
1          0
2          2
3          4
4          2
          ..
1048562    2
1048563    2
1048564    2
1048565    3
1048566    3
Name: Age, Length: 1045170, dtype: int64

In [99]:
new_df

,ID,CAB,Trans,Age,Gender
0,C5841053,3,0,2.0,1
1,C2142763,0,50,0.0,0
2,C4417068,3,7,2.0,1
3,C5342380,50,32,4.0,1
4,C9031234,1,27,2.0,1
...,...,...,...,...,...
1048562,C8020229,1,12,2.0,0
1048563,C6459278,5,7,2.0,0
1048564,C6412354,47,11,2.0,0
1048565,C6420483,2,15,3.0,0


In [100]:
new_df.describe()


,CAB,Trans,Age,Gender
count,1.045170e+06,1.045170e+06,1.045170e+06,1.045170e+06
mean,1.019271e+01,1.314349e+01,2.146427e+00,2.701647e-01
std,1.529700e+01,1.553733e+01,1.007966e+00,4.440449e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00
50%,3.000000e+00,7.000000e+00,2.000000e+00,0.000000e+00
75%,1.200000e+01,1.800000e+01,3.000000e+00,1.000000e+00
max,5.000000e+01,5.000000e+01,4.000000e+00,1.000000e+00


We have built but $Characteristic$-$Matrix$.
where the columns correspond to the users, while the rows represent the features. for each feature there will be a specific number of rows, which will correspond to a particular class of membership of that specific feature. the intervals defined for the features change from one to another as needed.

In [101]:
char_matrix = np.zeros((107 ,len(new_df)),dtype=('int')) # 107 = total classes
for i in tqdm(range(len(new_df)) ):
    char_matrix[new_df.iloc[i]['CAB'], i] = 1
    char_matrix[new_df.iloc[i]['Trans']+50, i] = 1
    char_matrix[int(new_df.iloc[i]['Age'])+100, i] = 1
    char_matrix[new_df.iloc[i]['Gender']+105, i] = 1
      

100%|██████████| 1045170/1045170 [06:34<00:00, 2648.29it/s]


In [102]:
np.save('char_matrix.npy', char_matrix )

In [11]:
C_M=np.load('char_matrix.npy')

In [12]:
C_M

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 0, 0]])

In [119]:
print(C_M.shape, C_M.size)

(107, 1045170) 111833190


In [13]:
from itertools import permutations

Create 20 permutation with lenght=107, the number of row of the $Characteristic$-$Matrix$.

In [15]:
h=list()
x=0
while x<20: 
    s=list(np.random.permutation(107))
    if s not in h:
        h.append(s)
        x+=1



In [ ]:
np.save('list_of_permutations.npy', h )

In [151]:
h=np.load('list_of_permutations.npy')

In [152]:
h

array([[ 97, 104,  74, ...,  41,  95,  77],
       [ 60,  61,  98, ...,  20,  27,   1],
       [  2, 106,  60, ...,  72,  45,  12],
       ...,
       [  8,  17,  36, ...,  79,  87,  40],
       [ 66,  73,  22, ...,  46, 103,  11],
       [ 43,  54,  72, ...,  41,  52,  98]])

Build signature matrix(g) with h. which is a list of 20 permutation of the $Characteristic$-$Matrix$ row's.

In [153]:
g=np.zeros((len(h) ,len(C_M[0])),dtype=('int'))
for i in tqdm(range(len(h))):  
    for s in range(len(C_M[0])):
        for j in range(len(h[0])): 
            r=h[i][j]
            if C_M[r][s] == 1:
                g[i][s]=j
                break
    


100%|██████████| 20/20 [05:06<00:00, 15.35s/it]


In [155]:
np.save('signature_matrix.npy', g )

In [157]:
g=np.load('signature_matrix.npy')
h=np.load('list_of_permutations.npy')

Let's build a list of dictionaries. Each dictionary consists of a certain number of buckets and refers to a suitable band.
Now given a query, its values will be transformed into ranges, a boolean vector will be created and through the previous permutations(h), its(the query's) minhash image will be created, it will be cut and compared(piece by piece) with the dictionary keys.
So since we have 4 bands, each one of 5 lines,
our minhash image will be cut into 4 parts by 5, and the first part will be matched with the keys of the first dictionary, the second with the keys of the second and so on. Found the corresponding keys we will have in the values of those keys the candidate transactions to be similar to our query.

In [158]:
b = 5 # number of bands
r = len(g)//b # number of row for each bands
buckets_list= []
for bands in tqdm(range(b)):
    dic={}
    #I check the list of documents just one time and I insert
    #the document in the dictionary checking if it exist or not
    for indice in range(len(g[0])):
        a=(g[0 + r*bands][indice],g[1 + r*bands][indice],g[2 + r*bands][indice],g[3 + r*bands][indice])
        if a in dic:
            dic[a].append(indice)
        else:
            dic[a]=[indice]
    buckets={}
    #I delete every items with only one element, 
    #it means that element has not similar documents
    for key, item in dic.items():
        if len(item)!=1:
            buckets[key]=item
    buckets_list.append(buckets)

100%|██████████| 5/5 [00:09<00:00,  1.91s/it]


In [ ]:
np.save('buckets_list.npy', buckets_list )

In [ ]:
buckets_list =np.load('buckets_list.npy')

In [160]:
#total number of buckets
c=0
for i in range(b):
    c+=len(buckets_list[i].keys())
print(c)    

25197


Query preprocessing:

In [161]:
df=df_.iloc[[0]] # take the firt row as a query 
df.DOB=pd.to_datetime(df.DOB)

/var/folders/cg/glf2wdrs1_q18pg9grzgpjqr0000gp/T/ipykernel_51737/221808561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.DOB=pd.to_datetime(df.DOB)


In [162]:
df

,ID,Gender,DOB,CABalance,TransA
0,C5841053,F,1994-10-01,17819.05,25.0


Now, given a query we want to find its candidates to be similar.
so it will be necessary to process the query in the same way we processed the elements of the dataset, find its images through the same permutations as before, and then compare them with the keys of our dictionaries, i.e. with the previously created buckets.

In [163]:
def query_process(df):
    # divide query row in class 
    
    q_df=df.copy()
    q_df.CABalance[0]=CAB_class(q_df.CABalance[0])
    q_df.TransA[0]=Trans_class(q_df.TransA[0]) 
    q_df.DOB[0]=age_class(age(q_df.DOB[0]))  
    q_df.Gender[0]=( 0 if q_df.Gender[0]=='M' else 1)

    # create boolean vector 
    vect= np.zeros((107 ,1),dtype=('int'))
    vect[int(q_df.iloc[0]['CABalance']), 0] = 1
    vect[int(q_df.iloc[0]['TransA'])+50, 0] = 1
    vect[int(q_df.iloc[0]['DOB'])+100, 0] = 1
    vect[q_df.iloc[0]['Gender']+105, 0] = 1
    
    # buid hash image of the query
    has = np.zeros((len(h) ,1),dtype=('int'))
    for i in range(len(h)):  
        for j in range(len(h[0])): 
            r=h[i][j]
            if vect[r][0] == 1:
                has[i][0]=j
                break
    return has 

In [ ]:
has=query_process(df)

In [166]:
candid=[]
for bas in range(b):
    a=(has[0 + r*bas][0],has[1 + r*bas][0],has[2 + r*bas][0],has[3 + r*bas][0])
    for kye, values in buckets_list[bas].items():
        if a == kye:
            candid.append(values)
            break
  

In [ ]:
candid[]

In [169]:
cand=list(set(candid[0]) & set(candid[1]) & set(candid[2]) & set(candid[3]) & set(candid[4]))


In [171]:
len(cand)


1693

In [172]:
for row in new_df.loc[cand]

,ID,CAB,Trans,Age,Gender
0,C5841053,3,0,2.0,1
163851,C6035218,10,4,2.0,0
401420,C6696444,1,3,3.0,0
835596,C1513559,0,7,2.0,1
876559,C1313440,5,19,3.0,0
...,...,...,...,...,...
524274,C1421916,1,0,2.0,0
40950,C2113811,1,9,2.0,0
573434,C3721662,0,5,2.0,0
876541,C8617652,23,4,3.0,0
